# Imports and Cleaning

In [68]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning, UndefinedMetricWarning

# Suppress specific warnings
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  
from sklearn.linear_model import RidgeClassifier  
from sklearn.naive_bayes import GaussianNB  
from sklearn.svm import SVC  
from sklearn.metrics import classification_report

In [71]:
from termcolor import colored as cl  # text customization
from sklearn.metrics import roc_auc_score  # AUC metric
from sklearn.linear_model import LogisticRegression  # Logistic Regression for classification
from sklearn.linear_model import RidgeClassifier  # Ridge Classifier for classification
from sklearn.naive_bayes import GaussianNB  # Naive Bayes for classification
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.preprocessing import label_binarize #as we have multi-class classification

In [72]:
#for random state (to have it consistent throughout the model)
seed = 9

In [73]:
data = pd.read_csv("../World_Datasets/final_dataset_world.csv")

In [74]:
data

,Country,S&P Rating,Country Name,country_code,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,unemployment_2019,unemployment_2020,unemployment_2021,unemployment_2022,unemployment_2023,current_account_balance_1990,current_account_balance_2000,current_account_balance_2014,current_account_balance_2015,current_account_balance_2016,current_account_balance_2017,current_account_balance_2018,current_account_balance_2019,current_account_balance_2020,current_account_balance_2021,current_account_balance_2022,current_account_balance_2023,exchange_rate_usd_1990,exchange_rate_usd_2000,exchange_rate_usd_2014,exchange_rate_usd_2015,exchange_rate_usd_2016,exchange_rate_usd_2017,exchange_rate_usd_2018,exchange_rate_usd_2019,exchange_rate_usd_2020,exchange_rate_usd_2021,exchange_rate_usd_2022,exchange_rate_usd_2023,gdp_1990,gdp_2000,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023,inflation_1990,inflation_2000,inflation_2014,inflation_2015,inflation_2016,inflation_2017,inflation_2018,inflation_2019,inflation_2020,inflation_2021,inflation_2022,inflation_2023,total_reserves_1990,total_reserves_2000,total_reserves_2014,total_reserves_2015,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,Unnamed: 73,value_counts
0,Australia,AAA,Australia,AUS,6.288000,6.078000,6.055000,5.711000,5.592000,5.300000,5.159000,6.456000,5.116000,3.701000,3.667000,-4.987491e+00,-3.721401e+00,-3.054734e+00,-4.229028e+00,-3.440490e+00,-2.722911e+00,-2.233402e+00,3.501218e-01,2.330931e+00,3.137209e+00,8.453239e-01,2.547373e-01,1.281057e+00,1.724827e+00,1.109363e+00,1.331090e+00,1.345214e+00,1.304758e+00,1.338412e+00,1.438507e+00,1.453085e+00,1.331224e+00,1.441664e+00,1.505191e+00,3.114267e+11,4.161678e+11,1.468265e+12,1.351296e+12,1.206837e+12,1.325583e+12,1.427809e+12,1.392724e+12,1.328414e+12,1.556736e+12,1.690858e+12,1.728057e+12,7.333022e+00,4.457435e+00,2.487923e+00,1.508367e+00,1.276991e+00,1.948647e+00,1.911401e+00,1.610768e+00,8.469055e-01,2.863910e+00,6.594097e+00,5.597015e+00,1.931874e+10,1.882155e+10,5.391033e+10,4.540604e+10,5.248079e+10,6.565372e+10,5.390953e+10,5.799469e+10,4.254463e+10,5.787750e+10,5.670190e+10,6.170335e+10,3.085167e+10,73.0
1,Canada,AAA,Canada,CAN,6.829000,7.023000,6.945000,7.038000,6.426000,5.837000,5.690000,9.657000,7.527000,5.280000,5.366000,-3.398803e+00,2.483228e+00,-2.318921e+00,-3.513987e+00,-3.093111e+00,-2.803427e+00,-2.385072e+00,-1.951703e+00,-2.010494e+00,1.277747e-02,-3.526320e-01,-7.295740e-01,1.166774e+00,1.485394e+00,1.104747e+00,1.278786e+00,1.325615e+00,1.297936e+00,1.295818e+00,1.326793e+00,1.341153e+00,1.253877e+00,1.301555e+00,1.349909e+00,5.960756e+11,7.447734e+11,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.743725e+12,1.655685e+12,2.007472e+12,2.161483e+12,2.142471e+12,4.780477e+00,2.719440e+00,1.906636e+00,1.125241e+00,1.428760e+00,1.596884e+00,2.268226e+00,1.949269e+00,7.169996e-01,3.395193e+00,6.802801e+00,3.879002e+00,2.352952e+10,3.242727e+10,7.469996e+10,7.975352e+10,8.271811e+10,8.667771e+10,8.392560e+10,8.529711e+10,9.042814e+10,1.066151e+11,1.069524e+11,1.175509e+11,5.877545e+10,73.0
2,Denmark,AAA,Denmark,DNK,4.476000,6.925000,6.278000,5.989000,5.833000,5.131000,5.018000,5.637000,5.043000,4.434000,5.142000,9.923083e-01,1.378800e+00,8.886178e+00,7.953938e+00,7.078509e+00,7.360245e+00,6.244573e+00,7.425270e+00,7.231130e+00,8.659145e+00,1.156562e+01,9.840846e+00,6.188558e+00,8.083144e+00,5.612467e+00,6.727907e+00,6.731718e+00,6.602893e+00,6.314619e+00,6.669447e+00,6.542152e+00,6.287113e+00,7.076152e+00,6.889703e+00,1.382177e+11,1.640438e+11,3.528326e+11,3.017589e+11,3.121818e+11,3.316106e+11,3.552934e+11,3.454015e+11,3.556310e+11,4.083782e+11,4.019456e+11,4.070919e+11,2.641603e+00,2.903282e+00,5.640205e-01,4.520342e-01,2.500000e-01,1.147132e+00,8.136095e-01,7.581316e-01,4.207120e-01,1.85

In [75]:
#dropping non-relevant columns
data = data.drop(['Unnamed: 73', 'value_counts', 'Country Name', 'country_code'], axis = 1)

In [76]:
print('Number of unique values: ', data['S&P Rating'].nunique())
print(data['S&P Rating'].unique())

Number of unique values:  38
['AAA' 'AA+' 'AA+\xa0' 'AA-' 'AA' 'AA\xa0' 'A+' 'NR' 'A\xa0' 'A' 'AA-\xa0'
 'A-' 'A-\xa0' 'BBB+' 'BBB' 'BBB\xa0' 'BBB+\xa0' 'BBB-' 'BBB-\xa0'
 'BB+\xa0' 'BB+' 'BB\xa0' 'BB' 'BB-\xa0' 'BB-' 'B+' 'B+\xa0' 'SD' 'B\xa0'
 'B-' 'B' 'B-\xa0' 'CCC+\xa0' 'CCC+' 'CCC' 'D' 'D\xa0' 'SD\xa0']


As 38 is too many values for a classification, we will regroup first, and drop the non-rated 'NR'.

In [77]:
data = data[data['S&P Rating'] != 'NR'] #dropping non-rated

In [78]:
data['target'] = np.where(data['S&P Rating'].str.match(r'.*A.*', na=False), 0, 
                          np.where(data['S&P Rating'].str.match(r'.*B.*', na=False), 1, 2))

We now have 3 classification possibilities. 

# Base Model

In [79]:
data

,Country,S&P Rating,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,unemployment_2019,unemployment_2020,unemployment_2021,unemployment_2022,unemployment_2023,current_account_balance_1990,current_account_balance_2000,current_account_balance_2014,current_account_balance_2015,current_account_balance_2016,current_account_balance_2017,current_account_balance_2018,current_account_balance_2019,current_account_balance_2020,current_account_balance_2021,current_account_balance_2022,current_account_balance_2023,exchange_rate_usd_1990,exchange_rate_usd_2000,exchange_rate_usd_2014,exchange_rate_usd_2015,exchange_rate_usd_2016,exchange_rate_usd_2017,exchange_rate_usd_2018,exchange_rate_usd_2019,exchange_rate_usd_2020,exchange_rate_usd_2021,exchange_rate_usd_2022,exchange_rate_usd_2023,gdp_1990,gdp_2000,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023,inflation_1990,inflation_2000,inflation_2014,inflation_2015,inflation_2016,inflation_2017,inflation_2018,inflation_2019,inflation_2020,inflation_2021,inflation_2022,inflation_2023,total_reserves_1990,total_reserves_2000,total_reserves_2014,total_reserves_2015,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,target
0,Australia,AAA,6.288000,6.078000,6.055000,5.711000,5.592000,5.300000,5.159000,6.456000,5.116000,3.701000,3.667000,-4.987491e+00,-3.721401e+00,-3.054734e+00,-4.229028e+00,-3.440490e+00,-2.722911e+00,-2.233402e+00,3.501218e-01,2.330931e+00,3.137209e+00,8.453239e-01,2.547373e-01,1.281057e+00,1.724827e+00,1.109363e+00,1.331090e+00,1.345214e+00,1.304758e+00,1.338412e+00,1.438507e+00,1.453085e+00,1.331224e+00,1.441664e+00,1.505191e+00,3.114267e+11,4.161678e+11,1.468265e+12,1.351296e+12,1.206837e+12,1.325583e+12,1.427809e+12,1.392724e+12,1.328414e+12,1.556736e+12,1.690858e+12,1.728057e+12,7.333022e+00,4.457435e+00,2.487923e+00,1.508367e+00,1.276991e+00,1.948647e+00,1.911401e+00,1.610768e+00,8.469055e-01,2.863910e+00,6.594097e+00,5.597015e+00,1.931874e+10,1.882155e+10,5.391033e+10,4.540604e+10,5.248079e+10,6.565372e+10,5.390953e+10,5.799469e+10,4.254463e+10,5.787750e+10,5.670190e+10,6.170335e+10,0
1,Canada,AAA,6.829000,7.023000,6.945000,7.038000,6.426000,5.837000,5.690000,9.657000,7.527000,5.280000,5.366000,-3.398803e+00,2.483228e+00,-2.318921e+00,-3.513987e+00,-3.093111e+00,-2.803427e+00,-2.385072e+00,-1.951703e+00,-2.010494e+00,1.277747e-02,-3.526320e-01,-7.295740e-01,1.166774e+00,1.485394e+00,1.104747e+00,1.278786e+00,1.325615e+00,1.297936e+00,1.295818e+00,1.326793e+00,1.341153e+00,1.253877e+00,1.301555e+00,1.349909e+00,5.960756e+11,7.447734e+11,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.743725e+12,1.655685e+12,2.007472e+12,2.161483e+12,2.142471e+12,4.780477e+00,2.719440e+00,1.906636e+00,1.125241e+00,1.428760e+00,1.596884e+00,2.268226e+00,1.949269e+00,7.169996e-01,3.395193e+00,6.802801e+00,3.879002e+00,2.352952e+10,3.242727e+10,7.469996e+10,7.975352e+10,8.271811e+10,8.667771e+10,8.392560e+10,8.529711e+10,9.042814e+10,1.066151e+11,1.069524e+11,1.175509e+11,0
2,Denmark,AAA,4.476000,6.925000,6.278000,5.989000,5.833000,5.131000,5.018000,5.637000,5.043000,4.434000,5.142000,9.923083e-01,1.378800e+00,8.886178e+00,7.953938e+00,7.078509e+00,7.360245e+00,6.244573e+00,7.425270e+00,7.231130e+00,8.659145e+00,1.156562e+01,9.840846e+00,6.188558e+00,8.083144e+00,5.612467e+00,6.727907e+00,6.731718e+00,6.602893e+00,6.314619e+00,6.669447e+00,6.542152e+00,6.287113e+00,7.076152e+00,6.889703e+00,1.382177e+11,1.640438e+11,3.528326e+11,3.017589e+11,3.121818e+11,3.316106e+11,3.552934e+11,3.454015e+11,3.556310e+11,4.083782e+11,4.019456e+11,4.070919e+11,2.641603e+00,2.903282e+00,5.640205e-01,4.520342e-01,2.500000e-01,1.147132e+00,8.136095e-01,7.581316e-01,4.207120e-01,1.853045e+00,7.696567e+00,3.305178e+00,1.122584e+10,1.569595e+10,7.539189e+10,6.518509e+10,6.421581e+10,7.524445e+10,

In [80]:
data = data.set_index(data['Country'])
data = data.drop('Country', axis = 1)

In [81]:
data.head()

,S&P Rating,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,unemployment_2019,unemployment_2020,unemployment_2021,unemployment_2022,unemployment_2023,current_account_balance_1990,current_account_balance_2000,current_account_balance_2014,current_account_balance_2015,current_account_balance_2016,current_account_balance_2017,current_account_balance_2018,current_account_balance_2019,current_account_balance_2020,current_account_balance_2021,current_account_balance_2022,current_account_balance_2023,exchange_rate_usd_1990,exchange_rate_usd_2000,exchange_rate_usd_2014,exchange_rate_usd_2015,exchange_rate_usd_2016,exchange_rate_usd_2017,exchange_rate_usd_2018,exchange_rate_usd_2019,exchange_rate_usd_2020,exchange_rate_usd_2021,exchange_rate_usd_2022,exchange_rate_usd_2023,gdp_1990,gdp_2000,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023,inflation_1990,inflation_2000,inflation_2014,inflation_2015,inflation_2016,inflation_2017,inflation_2018,inflation_2019,inflation_2020,inflation_2021,inflation_2022,inflation_2023,total_reserves_1990,total_reserves_2000,total_reserves_2014,total_reserves_2015,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,target
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,AAA,6.288000,6.078000,6.055000,5.711000,5.592000,5.300000,5.159000,6.456000,5.116000,3.70100,3.667000,-4.987491e+00,-3.721401e+00,-3.054734e+00,-4.229028e+00,-3.440490e+00,-2.722911e+00,-2.233402e+00,3.501218e-01,2.330931e+00,3.137209e+00,8.453239e-01,2.547373e-01,1.281057e+00,1.724827e+00,1.109363e+00,1.331090e+00,1.345214e+00,1.304758e+00,1.338412e+00,1.438507e+00,1.453085e+00,1.331224e+00,1.441664e+00,1.505191e+00,3.114267e+11,4.161678e+11,1.468265e+12,1.351296e+12,1.206837e+12,1.325583e+12,1.427809e+12,1.392724e+12,1.328414e+12,1.556736e+12,1.690858e+12,1.728057e+12,7.333022e+00,4.457435e+00,2.487923e+00,1.508367e+00,1.276991e+00,1.948647e+00,1.911401e+00,1.610768e+00,8.469055e-01,2.863910e+00,6.594097e+00,5.597015e+00,1.931874e+10,1.882155e+10,5.391033e+10,4.540604e+10,5.248079e+10,6.565372e+10,5.390953e+10,5.799469e+10,4.254463e+10,5.787750e+10,5.670190e+10,6.170335e+10,0
Canada,AAA,6.829000,7.023000,6.945000,7.038000,6.426000,5.837000,5.690000,9.657000,7.527000,5.28000,5.366000,-3.398803e+00,2.483228e+00,-2.318921e+00,-3.513987e+00,-3.093111e+00,-2.803427e+00,-2.385072e+00,-1.951703e+00,-2.010494e+00,1.277747e-02,-3.526320e-01,-7.295740e-01,1.166774e+00,1.485394e+00,1.104747e+00,1.278786e+00,1.325615e+00,1.297936e+00,1.295818e+00,1.326793e+00,1.341153e+00,1.253877e+00,1.301555e+00,1.349909e+00,5.960756e+11,7.447734e+11,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.743725e+12,1.655685e+12,2.007472e+12,2.161483e+12,2.142471e+12,4.780477e+00,2.719440e+00,1.906636e+00,1.125241e+00,1.428760e+00,1.596884e+00,2.268226e+00,1.949269e+00,7.169996e-01,3.395193e+00,6.802801e+00,3.879002e+00,2.352952e+10,3.242727e+10,7.469996e+10,7.975352e+10,8.271811e+10,8.667771e+10,8.392560e+10,8.529711e+10,9.042814e+10,1.066151e+11,1.069524e+11,1.175509e+11,0
Denmark,AAA,4.476000,6.925000,6.278000,5.989000,5.833000,5.131000,5.018000,5.637000,5.043000,4.43400,5.142000,9.923083e-01,1.378800e+00,8.886178e+00,7.953938e+00,7.078509e+00,7.360245e+00,6.244573e+00,7.425270e+00,7.231130e+00,8.659145e+00,1.156562e+01,9.840846e+00,6.188558e+00,8.083144e+00,5.612467e+00,6.727907e+00,6.731718e+00,6.602893e+00,6.314619e+00,6.669447e+00,6.542152e+00,6.287113e+00,7.076152e+00,6.889703e+00,1.382177e+11,1.640438e+11,3.528326e+11,3.017589e+11,3.121818e+11,3.316106e+11,3.552934e+11,3.454015e+11,3.556310e+11,4.083782e+11,4.019456e+11,4.070919e+11,2.641603e+00,2.903282e+00,5.640205e-01,4.520342e-01,2.500000e-01,1.147132e+00,8.136095e-01,7.581316e-01,4.207120e-01,1.853045e+00,7.696567e+00,3.305178e+00,1.122584e+10,1

## Train Test Split 

In [82]:
X = data.drop(['target', 'S&P Rating'], axis = 1)
y = data[['target']]

In [83]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=seed)

## Intial Modeling and Results 

In [84]:
# 1. Logistic Regression
logistic = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=seed)
logistic.fit(X_train, y_train)
logistic_test_yhat = logistic.predict(X_test)
logistic_train_yhat = logistic.predict(X_train)


# 2. Ridge Classifier
ridge_classifier = RidgeClassifier(alpha = 0.5, random_state=seed)
ridge_classifier.fit(X_train, y_train)
ridge_classifier_test_yhat = ridge_classifier.predict(X_test)
ridge_classifier_train_yhat = ridge_classifier.predict(X_train)

# 3. Gaussian Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
naive_bayes_test_yhat = naive_bayes.predict(X_test)
naive_bayes_train_yhat = naive_bayes.predict(X_train)

# 4. Support Vector Classifier
svc = SVC(probability = True, random_state=seed)
svc.fit(X_train, y_train)
svc_test_yhat = svc.predict(X_test)
svc_train_yhat = svc.predict(X_train)

In [85]:
# Binarize the output for multi-class AUC
y_train_binarized = label_binarize(y_train, classes=[0, 1, 2])  # Adjust class labels as needed
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2])

print(cl('AUC SCORE (Multi-Class):', attrs=['bold']))
print('-------------------------------------------------------------------------------')

# 1. Logistic Regression
logistic.fit(X_train, y_train)
logistic_train_auc = roc_auc_score(y_train_binarized, logistic.predict_proba(X_train), multi_class='ovr')
logistic_test_auc = roc_auc_score(y_test_binarized, logistic.predict_proba(X_test), multi_class='ovr')
print(cl('Logistic model:', attrs=['bold']))
print(cl('Train - AUC score is {:.4f}'.format(logistic_train_auc), attrs=['bold']))
print(cl('Test - AUC score is {:.4f}'.format(logistic_test_auc), attrs=['bold']))

print('-------------------------------------------------------------------------------')

# 2. Ridge Classifier
ridge_classifier.fit(X_train, y_train)
ridge_train_scores = ridge_classifier.decision_function(X_train)  # Multiclass decision scores
ridge_test_scores = ridge_classifier.decision_function(X_test)
ridge_classifier_train_auc = roc_auc_score(y_train_binarized, ridge_train_scores, multi_class='ovr')
ridge_classifier_test_auc = roc_auc_score(y_test_binarized, ridge_test_scores, multi_class='ovr')
print(cl('Ridge model:', attrs=['bold']))
print(cl('Train - AUC score is {:.4f}'.format(ridge_classifier_train_auc), attrs=['bold']))
print(cl('Test - AUC score is {:.4f}'.format(ridge_classifier_test_auc), attrs=['bold']))

print('-------------------------------------------------------------------------------')

# 3. Gaussian Naive Bayes
naive_bayes.fit(X_train, y_train)
naive_bayes_train_auc = roc_auc_score(y_train_binarized, naive_bayes.predict_proba(X_train), multi_class='ovr')
naive_bayes_test_auc = roc_auc_score(y_test_binarized, naive_bayes.predict_proba(X_test), multi_class='ovr')
print(cl('Naive Bayes model:', attrs=['bold']))
print(cl('Train - AUC score is {:.4f}'.format(naive_bayes_train_auc), attrs=['bold']))
print(cl('Test - AUC score is {:.4f}'.format(naive_bayes_test_auc), attrs=['bold']))

print('-------------------------------------------------------------------------------')

# 4. Support Vector Classifier
svc.fit(X_train, y_train)
svc_train_auc = roc_auc_score(y_train_binarized, svc.decision_function(X_train), multi_class='ovr')  # Use decision_function
svc_test_auc = roc_auc_score(y_test_binarized, svc.decision_function(X_test), multi_class='ovr')
print(cl('SVC model:', attrs=['bold']))
print(cl('Train - AUC score is {:.4f}'.format(svc_train_auc), attrs=['bold']))
print(cl('Test - AUC score is {:.4f}'.format(svc_test_auc), attrs=['bold']))



AUC SCORE (Multi-Class):
-------------------------------------------------------------------------------
Logistic model:
Train - AUC score is 0.9148
Test - AUC score is 0.8102
-------------------------------------------------------------------------------
Ridge model:
Train - AUC score is 1.0000
Test - AUC score is 0.4185
-------------------------------------------------------------------------------
Naive Bayes model:
Train - AUC score is 0.6967
Test - AUC score is 0.7161
-------------------------------------------------------------------------------
SVC model:
Train - AUC score is 0.7028
Test - AUC score is 0.5403


In [86]:
print(cl('Logistic Model: Other Metrics:', attrs=['bold']))
# 1. Logistic Regression
logistic.fit(X_train, y_train)
print('     ')
print(cl('Classification Report (Test):'))
print(classification_report(y_test, logistic_test_yhat))

Logistic Model: Other Metrics:
     
Classification Report (Test):
              precision    recall  f1-score   support

           0       0.70      0.61      0.65        23
           1       0.75      0.67      0.71        27
           2       0.14      1.00      0.25         1

    accuracy                           0.65        51
   macro avg       0.53      0.76      0.54        51
weighted avg       0.72      0.65      0.67        51



In [87]:
print(cl('Ridge Classifier: Other Metrics:', attrs=['bold']))
# 2. Ridge Classifier
ridge_classifier.fit(X_train, y_train)
print('     ')
print(cl('Classification Report (Test):'))
print(classification_report(y_test, ridge_classifier_test_yhat))

Ridge Classifier: Other Metrics:
     
Classification Report (Test):
              precision    recall  f1-score   support

           0       0.62      0.35      0.44        23
           1       0.63      0.44      0.52        27
           2       0.00      0.00      0.00         1

    accuracy                           0.39        51
   macro avg       0.42      0.26      0.32        51
weighted avg       0.61      0.39      0.48        51



In [88]:
print(cl('Gaussian Naive Bayes: Other Metrics:', attrs=['bold']))
# 3. Gaussian Naive Bayes
naive_bayes.fit(X_train, y_train)
print('     ')
print(cl('Classification Report (Test):'))
print(classification_report(y_test, naive_bayes_test_yhat))

Gaussian Naive Bayes: Other Metrics:
     
Classification Report (Test):
              precision    recall  f1-score   support

           0       0.83      0.22      0.34        23
           1       0.57      0.85      0.69        27
           2       0.20      1.00      0.33         1

    accuracy                           0.57        51
   macro avg       0.54      0.69      0.45        51
weighted avg       0.68      0.57      0.53        51



In [89]:
print(cl('Support Vector Classifier: Other Metrics:', attrs=['bold']))
# 4. Support Vector Classifier
svc.fit(X_train, y_train)
print('     ')
print(cl('Classification Report (Test):'))
print(classification_report(y_test,svc_test_yhat))

Support Vector Classifier: Other Metrics:
     
Classification Report (Test):
              precision    recall  f1-score   support

           0       0.80      0.17      0.29        23
           1       0.57      0.96      0.71        27
           2       0.00      0.00      0.00         1

    accuracy                           0.59        51
   macro avg       0.46      0.38      0.33        51
weighted avg       0.66      0.59      0.51        51



In [90]:
# AUC scores from all models
model_names = ['Logistic Regression', 'Ridge Classifier', 'Naive Bayes', 'SVC']
train_auc_scores = [logistic_train_auc, ridge_classifier_train_auc, naive_bayes_train_auc, svc_train_auc]
test_auc_scores = [logistic_test_auc, ridge_classifier_test_auc, naive_bayes_test_auc, svc_test_auc]

# Create a DataFrame for visualization
auc_data = pd.DataFrame({
    'Model': model_names * 2,
    'Dataset': ['Train'] * len(model_names) + ['Test'] * len(model_names),
    'AUC Score': train_auc_scores + test_auc_scores})

# Create the bar plot with the plotly_dark template
fig = px.bar(
    auc_data, 
    x='Model', 
    y='AUC Score', 
    color='Dataset', 
    barmode='group',
    title='Comparison of AUC Scores Across Models',
    labels={'AUC Score': 'AUC Score', 'Model': 'Classification Model'},
    template='plotly_dark'
)

# Customize layout
fig.update_layout(
    title_font_size=20,
    xaxis_title='Model',
    yaxis_title='AUC Score',
    legend_title='Dataset',
)

# Display the plot
fig.show()


# Improved Modeling

## Feature Engineering 

Creation of relevant ratios recarding country ratings. 

In [91]:
df = data.copy()

In [92]:
df.head()

,S&P Rating,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,unemployment_2019,unemployment_2020,unemployment_2021,unemployment_2022,unemployment_2023,current_account_balance_1990,current_account_balance_2000,current_account_balance_2014,current_account_balance_2015,current_account_balance_2016,current_account_balance_2017,current_account_balance_2018,current_account_balance_2019,current_account_balance_2020,current_account_balance_2021,current_account_balance_2022,current_account_balance_2023,exchange_rate_usd_1990,exchange_rate_usd_2000,exchange_rate_usd_2014,exchange_rate_usd_2015,exchange_rate_usd_2016,exchange_rate_usd_2017,exchange_rate_usd_2018,exchange_rate_usd_2019,exchange_rate_usd_2020,exchange_rate_usd_2021,exchange_rate_usd_2022,exchange_rate_usd_2023,gdp_1990,gdp_2000,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023,inflation_1990,inflation_2000,inflation_2014,inflation_2015,inflation_2016,inflation_2017,inflation_2018,inflation_2019,inflation_2020,inflation_2021,inflation_2022,inflation_2023,total_reserves_1990,total_reserves_2000,total_reserves_2014,total_reserves_2015,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,target
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,AAA,6.288000,6.078000,6.055000,5.711000,5.592000,5.300000,5.159000,6.456000,5.116000,3.70100,3.667000,-4.987491e+00,-3.721401e+00,-3.054734e+00,-4.229028e+00,-3.440490e+00,-2.722911e+00,-2.233402e+00,3.501218e-01,2.330931e+00,3.137209e+00,8.453239e-01,2.547373e-01,1.281057e+00,1.724827e+00,1.109363e+00,1.331090e+00,1.345214e+00,1.304758e+00,1.338412e+00,1.438507e+00,1.453085e+00,1.331224e+00,1.441664e+00,1.505191e+00,3.114267e+11,4.161678e+11,1.468265e+12,1.351296e+12,1.206837e+12,1.325583e+12,1.427809e+12,1.392724e+12,1.328414e+12,1.556736e+12,1.690858e+12,1.728057e+12,7.333022e+00,4.457435e+00,2.487923e+00,1.508367e+00,1.276991e+00,1.948647e+00,1.911401e+00,1.610768e+00,8.469055e-01,2.863910e+00,6.594097e+00,5.597015e+00,1.931874e+10,1.882155e+10,5.391033e+10,4.540604e+10,5.248079e+10,6.565372e+10,5.390953e+10,5.799469e+10,4.254463e+10,5.787750e+10,5.670190e+10,6.170335e+10,0
Canada,AAA,6.829000,7.023000,6.945000,7.038000,6.426000,5.837000,5.690000,9.657000,7.527000,5.28000,5.366000,-3.398803e+00,2.483228e+00,-2.318921e+00,-3.513987e+00,-3.093111e+00,-2.803427e+00,-2.385072e+00,-1.951703e+00,-2.010494e+00,1.277747e-02,-3.526320e-01,-7.295740e-01,1.166774e+00,1.485394e+00,1.104747e+00,1.278786e+00,1.325615e+00,1.297936e+00,1.295818e+00,1.326793e+00,1.341153e+00,1.253877e+00,1.301555e+00,1.349909e+00,5.960756e+11,7.447734e+11,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.743725e+12,1.655685e+12,2.007472e+12,2.161483e+12,2.142471e+12,4.780477e+00,2.719440e+00,1.906636e+00,1.125241e+00,1.428760e+00,1.596884e+00,2.268226e+00,1.949269e+00,7.169996e-01,3.395193e+00,6.802801e+00,3.879002e+00,2.352952e+10,3.242727e+10,7.469996e+10,7.975352e+10,8.271811e+10,8.667771e+10,8.392560e+10,8.529711e+10,9.042814e+10,1.066151e+11,1.069524e+11,1.175509e+11,0
Denmark,AAA,4.476000,6.925000,6.278000,5.989000,5.833000,5.131000,5.018000,5.637000,5.043000,4.43400,5.142000,9.923083e-01,1.378800e+00,8.886178e+00,7.953938e+00,7.078509e+00,7.360245e+00,6.244573e+00,7.425270e+00,7.231130e+00,8.659145e+00,1.156562e+01,9.840846e+00,6.188558e+00,8.083144e+00,5.612467e+00,6.727907e+00,6.731718e+00,6.602893e+00,6.314619e+00,6.669447e+00,6.542152e+00,6.287113e+00,7.076152e+00,6.889703e+00,1.382177e+11,1.640438e+11,3.528326e+11,3.017589e+11,3.121818e+11,3.316106e+11,3.552934e+11,3.454015e+11,3.556310e+11,4.083782e+11,4.019456e+11,4.070919e+11,2.641603e+00,2.903282e+00,5.640205e-01,4.520342e-01,2.500000e-01,1.147132e+00,8.136095e-01,7.581316e-01,4.207120e-01,1.853045e+00,7.696567e+00,3.305178e+00,1.122584e+10,1

In [93]:
df.columns

Index(['S&P Rating', 'unemployment_2000', 'unemployment_2014',
       'unemployment_2015', 'unemployment_2016', 'unemployment_2017',
       'unemployment_2018', 'unemployment_2019', 'unemployment_2020',
       'unemployment_2021', 'unemployment_2022', 'unemployment_2023',
       'current_account_balance_1990', 'current_account_balance_2000',
       'current_account_balance_2014', 'current_account_balance_2015',
       'current_account_balance_2016', 'current_account_balance_2017',
       'current_account_balance_2018', 'current_account_balance_2019',
       'current_account_balance_2020', 'current_account_balance_2021',
       'current_account_balance_2022', 'current_account_balance_2023',
       'exchange_rate_usd_1990', 'exchange_rate_usd_2000',
       'exchange_rate_usd_2014', 'exchange_rate_usd_2015',
       'exchange_rate_usd_2016', 'exchange_rate_usd_2017',
       'exchange_rate_usd_2018', 'exchange_rate_usd_2019',
       'exchange_rate_usd_2020', 'exchange_rate_usd_2021',
      

#### Current Account Balance to GDP Ratio

In [94]:
# years for which we have data
years = [1990, 2000, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

for year in years:
    cab_col = f'current_account_balance_{year}'
    gdp_col = f'gdp_{year}'
    ratio_col = f'cab_to_gdp_{year}'
    
    # Ensure the columns exist before creating the ratio
    if cab_col in df.columns and gdp_col in df.columns:
        df[ratio_col] = df[cab_col] / df[gdp_col]

#### Total reserves to GDP Ratio

In [95]:
for year in years:
    reserves_col = f'total_reserves_{year}'
    gdp_col = f'gdp_{year}'
    ratio_col = f'reserves_to_gdp_{year}'
    
    if reserves_col in df.columns and gdp_col in df.columns:
        df[ratio_col] = df[reserves_col] / df[gdp_col]

#### GDP Growth Rate 

In [96]:
#only the years for which we can calculate a contiguous change
growth_years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

for i in range(1, len(growth_years)):
    previous = growth_years[i - 1]
    current = growth_years[i]
    gdp_prev = f'gdp_{previous}'
    gdp_curr = f'gdp_{current}'
    growth_col = f'gdp_growth_{previous}_{current}'
    
    if gdp_prev in df.columns and gdp_curr in df.columns:
        df[growth_col] = (df[gdp_curr] - df[gdp_prev]) / df[gdp_prev]

### Inflation to Unemployment Ratio

In [97]:
for year in years:
    inflation_col = f'inflation_{year}'
    unemployment_col = f'unemployment_{year}'
    ratio_col = f'inflation_to_unemployment_{year}'
    
    if inflation_col in df.columns and unemployment_col in df.columns:
        df[ratio_col] = df[inflation_col] / df[unemployment_col]

In [98]:
df

,S&P Rating,unemployment_2000,unemployment_2014,unemployment_2015,unemployment_2016,unemployment_2017,unemployment_2018,unemployment_2019,unemployment_2020,unemployment_2021,unemployment_2022,unemployment_2023,current_account_balance_1990,current_account_balance_2000,current_account_balance_2014,current_account_balance_2015,current_account_balance_2016,current_account_balance_2017,current_account_balance_2018,current_account_balance_2019,current_account_balance_2020,current_account_balance_2021,current_account_balance_2022,current_account_balance_2023,exchange_rate_usd_1990,exchange_rate_usd_2000,exchange_rate_usd_2014,exchange_rate_usd_2015,exchange_rate_usd_2016,exchange_rate_usd_2017,exchange_rate_usd_2018,exchange_rate_usd_2019,exchange_rate_usd_2020,exchange_rate_usd_2021,exchange_rate_usd_2022,exchange_rate_usd_2023,gdp_1990,gdp_2000,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,gdp_2019,gdp_2020,gdp_2021,gdp_2022,gdp_2023,inflation_1990,inflation_2000,inflation_2014,inflation_2015,inflation_2016,inflation_2017,inflation_2018,inflation_2019,inflation_2020,inflation_2021,inflation_2022,inflation_2023,total_reserves_1990,total_reserves_2000,total_reserves_2014,total_reserves_2015,total_reserves_2016,total_reserves_2017,total_reserves_2018,total_reserves_2019,total_reserves_2020,total_reserves_2021,total_reserves_2022,total_reserves_2023,target,cab_to_gdp_1990,cab_to_gdp_2000,cab_to_gdp_2014,cab_to_gdp_2015,cab_to_gdp_2016,cab_to_gdp_2017,cab_to_gdp_2018,cab_to_gdp_2019,cab_to_gdp_2020,cab_to_gdp_2021,cab_to_gdp_2022,cab_to_gdp_2023,reserves_to_gdp_1990,reserves_to_gdp_2000,reserves_to_gdp_2014,reserves_to_gdp_2015,reserves_to_gdp_2016,reserves_to_gdp_2017,reserves_to_gdp_2018,reserves_to_gdp_2019,reserves_to_gdp_2020,reserves_to_gdp_2021,reserves_to_gdp_2022,reserves_to_gdp_2023,gdp_growth_2014_2015,gdp_growth_2015_2016,gdp_growth_2016_2017,gdp_growth_2017_2018,gdp_growth_2018_2019,gdp_growth_2019_2020,gdp_growth_2020_2021,gdp_growth_2021_2022,gdp_growth_2022_2023,inflation_to_unemployment_2000,inflation_to_unemployment_2014,inflation_to_unemployment_2015,inflation_to_unemployment_2016,inflation_to_unemployment_2017,inflation_to_unemployment_2018,inflation_to_unemployment_2019,inflation_to_unemployment_2020,inflation_to_unemployment_2021,inflation_to_unemployment_2022,inflation_to_unemployment_2023
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,AAA,6.288000,6.078000,6.055000,5.711000,5.592000,5.300000,5.159000,6.456000,5.116000,3.701000,3.667000,-4.987491e+00,-3.721401e+00,-3.054734e+00,-4.229028e+00,-3.440490e+00,-2.722911e+00,-2.233402e+00,3.501218e-01,2.330931e+00,3.137209e+00,8.453239e-01,2.547373e-01,1.281057e+00,1.724827e+00,1.109363e+00,1.331090e+00,1.345214e+00,1.304758e+00,1.338412e+00,1.438507e+00,1.453085e+00,1.331224e+00,1.441664e+00,1.505191e+00,3.114267e+11,4.161678e+11,1.468265e+12,1.351296e+12,1.206837e+12,1.325583e+12,1.427809e+12,1.392724e+12,1.328414e+12,1.556736e+12,1.690858e+12,1.728057e+12,7.333022e+00,4.457435e+00,2.487923e+00,1.508367e+00,1.276991e+00,1.948647e+00,1.911401e+00,1.610768e+00,8.469055e-01,2.863910e+00,6.594097e+00,5.597015e+00,1.931874e+10,1.882155e+10,5.391033e+10,4.540604e+10,5.248079e+10,6.565372e+10,5.390953e+10,5.799469e+10,4.254463e+10,5.787750e+10,5.670190e+10,6.170335e+10,0,-1.601498e-11,-8.942069e-12,-2.080506e-12,-3.129608e-12,-2.850832e-12,-2.054124e-12,-1.564216e-12,2.513936e-13,1.754672e-12,2.015248e-12,4.999378e-13,1.474125e-13,0.062033,0.045226,0.036717,0.033602,0.043486,0.049528,0.037757,0.041641,0.032027,0.037179,0.033534,0.035707,-0.079665,-0.106904,0.098394,0.077118,-0.024573,-0.046176,0.171875,0.086156,0.022000,7.088796e-01,4.093325e-01,2.491109e-01,2.236020e-01,3.484706e-01,3.606417e-01,3.122248e-01,1.311812e-01,5.597948e-01,1.781707e+00,1.526320e+00
Canada,AAA,6.829000,7.023000,6.945000,7.038000,6.426000,5.837000,5.690000,9.657000,7.527000,5.280000,5.366000,-3.398803e+00,2.48322

## Correlations maps

In [99]:
df = df.drop('S&P Rating', axis = 1)

In [100]:
# Variables to compare
variables_to_compare = [
    'unemployment_2000', 'unemployment_2014', 'unemployment_2015', 'unemployment_2016',
    'unemployment_2017', 'unemployment_2018', 'unemployment_2019', 'unemployment_2020',
    'unemployment_2021', 'unemployment_2022', 'unemployment_2023'
]

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Unemployment Variables", 
                template='plotly_dark', 
                width=1000, 
                height=600)  

fig.show()


In [ ]:
# Variables to compare
variables_to_compare = ['current_account_balance_1990', 'current_account_balance_2000',
       'current_account_balance_2014', 'current_account_balance_2015',
       'current_account_balance_2016', 'current_account_balance_2017',
       'current_account_balance_2018', 'current_account_balance_2019',
       'current_account_balance_2020', 'current_account_balance_2021',
       'current_account_balance_2022', 'current_account_balance_2023'
]

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Current Account Balance Variables", 
                template='plotly_dark', 
                width=1000, 
                height=600) 

fig.show()


In [102]:
# Variables to compare
variables_to_compare = ['exchange_rate_usd_1990', 'exchange_rate_usd_2000',
       'exchange_rate_usd_2014', 'exchange_rate_usd_2015',
       'exchange_rate_usd_2016', 'exchange_rate_usd_2017',
       'exchange_rate_usd_2018', 'exchange_rate_usd_2019',
       'exchange_rate_usd_2020', 'exchange_rate_usd_2021',
       'exchange_rate_usd_2022', 'exchange_rate_usd_2023',
]

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Exchange Rate Variables", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [ ]:
# Variables to compare
variables_to_compare = ['inflation_1990', 'inflation_2000', 'inflation_2014', 'inflation_2015',
       'inflation_2016', 'inflation_2017', 'inflation_2018', 'inflation_2019',
       'inflation_2020', 'inflation_2021', 'inflation_2022', 'inflation_2023'
]

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Inflation Variables", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [ ]:
# Variables to compare
variables_to_compare = ['gdp_1990',
       'gdp_2000', 'gdp_2014', 'gdp_2015', 'gdp_2016', 'gdp_2017', 'gdp_2018',
       'gdp_2019', 'gdp_2020', 'gdp_2021', 'gdp_2022', 'gdp_2023'
]

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs GDP Variables", 
                template='plotly_dark', 
                width=1000, 
                height=600) 

fig.show()


In [105]:
# Variables to compare
variables_to_compare = ['total_reserves_1990', 'total_reserves_2000', 'total_reserves_2014',
       'total_reserves_2015', 'total_reserves_2016', 'total_reserves_2017',
       'total_reserves_2018', 'total_reserves_2019', 'total_reserves_2020',
       'total_reserves_2021', 'total_reserves_2022', 'total_reserves_2023'
]

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Total Reserves Variables", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [106]:
# Variables to compare
variables_to_compare = [ 'inflation_2023', 'gdp_1990', 'gdp_2023', 'unemployment_2023', 'current_account_balance_2023',
                        'total_reserves_1990', 'total_reserves_2023']

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs 2023 (and 1990) Variables", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [107]:
# Variables to compare
variables_to_compare = [ 'cab_to_gdp_1990', 'cab_to_gdp_2000', 'cab_to_gdp_2014', 'cab_to_gdp_2015', 'cab_to_gdp_2016', 
                        'cab_to_gdp_2017', 'cab_to_gdp_2018', 'cab_to_gdp_2019', 'cab_to_gdp_2020', 'cab_to_gdp_2021', 
                        'cab_to_gdp_2022', 'cab_to_gdp_2023']

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Current Account Balance to GDP Ratio", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [108]:
# Variables to compare
variables_to_compare = [ 'reserves_to_gdp_1990', 'reserves_to_gdp_2000', 'reserves_to_gdp_2014', 'reserves_to_gdp_2015', 
                        'reserves_to_gdp_2016', 'reserves_to_gdp_2017', 'reserves_to_gdp_2018', 'reserves_to_gdp_2019', 
                        'reserves_to_gdp_2020', 'reserves_to_gdp_2021', 'reserves_to_gdp_2022', 'reserves_to_gdp_2023']

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Reserves to GDP Ratio", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [109]:
# Variables to compare
variables_to_compare = [ 'gdp_growth_2014_2015', 'gdp_growth_2015_2016', 'gdp_growth_2016_2017', 'gdp_growth_2017_2018', 
                        'gdp_growth_2018_2019', 'gdp_growth_2019_2020', 'gdp_growth_2020_2021', 'gdp_growth_2021_2022', 
                        'gdp_growth_2022_2023']

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()


fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs GDP Growth", 
                template='plotly_dark', 
                width=1000,  
                height=600)  

fig.show()


In [111]:
# Variables to compare
variables_to_compare = [ 'inflation_to_unemployment_2000', 'inflation_to_unemployment_2014', 'inflation_to_unemployment_2015', 
                        'inflation_to_unemployment_2016', 'inflation_to_unemployment_2017', 'inflation_to_unemployment_2018', 
                        'inflation_to_unemployment_2019', 'inflation_to_unemployment_2020', 'inflation_to_unemployment_2021', 
                        'inflation_to_unemployment_2022', 'inflation_to_unemployment_2023']

# Compute the correlation matrix for 'target' and the selected variables
columns_to_use = ['target'] + variables_to_compare
df_corr_matrix = df[columns_to_use].corr()

# Take the absolute values of the correlation matrix
df_corr_matrix_abs = df_corr_matrix.abs()

fig = px.imshow(df_corr_matrix_abs, 
                color_continuous_scale='RdBu_r', 
                title="Correlation Heatmap: Target vs Inflation to Unemployment Ratio", 
                template='plotly_dark', 
                width=1000, 
                height=600)  

fig.show()


In [110]:
column_list = df.columns.tolist()
print(column_list)

['unemployment_2000', 'unemployment_2014', 'unemployment_2015', 'unemployment_2016', 'unemployment_2017', 'unemployment_2018', 'unemployment_2019', 'unemployment_2020', 'unemployment_2021', 'unemployment_2022', 'unemployment_2023', 'current_account_balance_1990', 'current_account_balance_2000', 'current_account_balance_2014', 'current_account_balance_2015', 'current_account_balance_2016', 'current_account_balance_2017', 'current_account_balance_2018', 'current_account_balance_2019', 'current_account_balance_2020', 'current_account_balance_2021', 'current_account_balance_2022', 'current_account_balance_2023', 'exchange_rate_usd_1990', 'exchange_rate_usd_2000', 'exchange_rate_usd_2014', 'exchange_rate_usd_2015', 'exchange_rate_usd_2016', 'exchange_rate_usd_2017', 'exchange_rate_usd_2018', 'exchange_rate_usd_2019', 'exchange_rate_usd_2020', 'exchange_rate_usd_2021', 'exchange_rate_usd_2022', 'exchange_rate_usd_2023', 'gdp_1990', 'gdp_2000', 'gdp_2014', 'gdp_2015', 'gdp_2016', 'gdp_2017', 

## Feature Selection 

Our goal is to remove redundant and irrelevant variables from our dataframe to avoid having too much noise and use too much computational power for nothing. 
As we have many values with a correlation with the target below 0.3, we will put a low threshold for irrelevance. 

In [122]:
irrelevance_threshold = 0.05
redundancy_threshold = 0.95

In [123]:
# Remove irrelevant variables
correlations_with_target = df.corr()['target'].abs()  # Absolute correlations with the target
relevant_vars = correlations_with_target[correlations_with_target >= irrelevance_threshold].index.tolist()

# Filter the dataframe to include only relevant variables
filtered_df = df[relevant_vars]

In [124]:
# Remove redundant variables (highly correlated with each other, but not with the target)
correlation_matrix = filtered_df.corr().abs()  # Compute absolute correlation matrix
np.fill_diagonal(correlation_matrix.values, 0)  # Fill diagonal with 0 to ignore self-correlation

# Keep track of variables to drop
variables_to_drop = set()

# Iterate through variables to find redundancies
for col in correlation_matrix.columns:
    if col not in variables_to_drop:
        # Find variables highly correlated with the current variable
        redundant_vars = correlation_matrix[col][correlation_matrix[col] > redundancy_threshold].index.tolist()
        # For redundant variables, keep the one most correlated with the target
        for redundant_var in redundant_vars:
            if redundant_var != col:
                # Compare correlation with target
                if correlations_with_target[col] >= correlations_with_target[redundant_var]:
                    variables_to_drop.add(redundant_var)  # Drop the redundant variable
                else:
                    variables_to_drop.add(col)

# Final list of variables after removing irrelevant and redundant variables
final_vars = [var for var in filtered_df.columns if var not in variables_to_drop]

# Create the final dataframe
final_df = df[final_vars]


In [125]:
final_df

,unemployment_2000,unemployment_2022,exchange_rate_usd_1990,exchange_rate_usd_2023,gdp_1990,gdp_2014,inflation_1990,total_reserves_1990,total_reserves_2000,total_reserves_2021,target,cab_to_gdp_2014,cab_to_gdp_2018,reserves_to_gdp_1990,reserves_to_gdp_2000,reserves_to_gdp_2017,gdp_growth_2014_2015,gdp_growth_2015_2016,gdp_growth_2016_2017,gdp_growth_2017_2018,gdp_growth_2018_2019,gdp_growth_2019_2020,gdp_growth_2020_2021,gdp_growth_2021_2022,inflation_to_unemployment_2023
Country,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,6.288000,3.701000,1.281057e+00,1.505191e+00,3.114267e+11,1.468265e+12,7.333022e+00,1.931874e+10,1.882155e+10,5.787750e+10,0,-2.080506e-12,-1.564216e-12,0.062033,0.045226,0.049528,-0.079665,-0.106904,0.098394,0.077118,-0.024573,-0.046176,0.171875,0.086156,1.526320e+00
Canada,6.829000,5.280000,1.166774e+00,1.349909e+00,5.960756e+11,1.805750e+12,4.780477e+00,2.352952e+10,3.242727e+10,1.066151e+11,0,-1.284187e-12,-1.382386e-12,0.039474,0.043540,0.052555,-0.138026,-0.018319,0.079366,0.046120,0.010662,-0.050490,0.212472,0.076719,7.228851e-01
Denmark,4.476000,4.434000,6.188558e+00,6.889703e+00,1.382177e+11,3.528326e+11,2.641603e+00,1.122584e+10,1.569595e+10,8.223584e+10,0,2.518525e-11,1.757582e-11,0.081219,0.095681,0.226906,-0.144753,0.034541,0.062235,0.071417,-0.027842,0.029616,0.148320,-0.015752,6.427807e-01
Germany,7.917000,3.135000,1.615733e+00,1.629982e+12,1.778162e+12,3.965801e+12,2.696468e+00,1.045473e+11,8.749687e+10,2.957362e+11,0,1.813629e-12,2.084088e-12,0.058795,0.044483,0.053143,-0.136727,0.033362,0.063686,0.076776,-0.023396,-0.004313,0.103589,-0.042477,1.952853e+00
Liechtenstein,9.640537,8.230760,1.840786e+08,2.638014e+10,1.421509e+09,6.657527e+09,6.816208e+09,3.544428e+09,3.271780e+09,1.090593e+09,0,6.469366e-03,1.487165e-02,2.493426,1.317200,0.336899,-0.058432,-0.004979,0.037998,0.033720,-0.038274,-0.004754,0.203643,-0.045248,4.791054e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sri Lanka,7.740000,6.330000,4.006292e+01,3.275065e+02,8.032551e+09,8.252854e+10,2.149525e+01,4.470305e+08,1.131355e+09,3.136992e+09,2,-2.918362e-11,-3.134757e-11,0.055652,0.069277,0.084333,0.031655,0.033724,0.072308,0.001246,-0.057981,-0.052920,0.051065,-0.163239,2.600814e+00
Ukraine,11.707000,8.411791,1.952346e-02,3.657381e+01,8.139356e+10,1.335039e+11,8.937851e+10,7.386012e+08,1.477202e+09,3.096667e+10,2,-2.578649e-11,-3.754273e-11,0.009074,0.045628,0.167819,-0.318140,0.025540,0.200680,0.167727,0.175657,0.017771,0.275500,-0.189103,1.838311e+00
Lebanon,8.594000,11.599000,6.950892e+02,1.419243e+09,2.838485e+09,4.809521e+10,1.049621e+10,4.210389e+09,8.474638e+09,3.523922e+10,2,-5.453737e-10,-4.434675e-10,1.483323,0.490988,1.044955,0.038135,0.024394,0.036764,0.035337,-0.060027,-0.385495,-0.270565,-0.092492,1.913892e+01
